# transformers: Bidirectional LMs

In [ ]:
import torch
from transformers import (
    set_seed,
    AutoTokenizer,
    AutoModel, # feature extractors (without task-specific head)
    AutoModelForSequenceClassification # sequence classification (with final head)
    # AutoModelForMaskedLM # bidirectional models (BERT-like, encoder-only)
)

In [ ]:
# set random seed manually
set_seed(123)

## Load base model

In [ ]:
# set model name
base_model_name = 'distilbert/distilbert-base-uncased'

In [ ]:
# create tokenizer
base_tokenizer = AutoTokenizer.from_pretrained(base_model_name)

print(base_tokenizer)

In [ ]:
# load model
base_model = AutoModel.from_pretrained(
    base_model_name,
    device_map='cpu',
    torch_dtype=torch.bfloat16 # use brain floating point format
)

base_model = base_model.eval()

print('Model device: {}'.format(base_model.device))
print('Model dtype: {}'.format(base_model.dtype))
print('Memory footprint: {:.2f} GiB'.format(base_model.get_memory_footprint() * 1e-9))

print(base_model)

## Compute embeddings

In [ ]:
# create raw input
raw_input = [
    'A rabbit goes into the supermarket.',
    'One, two, three, four, five.'
]

# tokenize
model_input = base_tokenizer(
    raw_input,
    add_special_tokens=True, # add CLS token at the beginning
    padding=True, # turn on padding (for batched inputs)
    truncation=True, # turn on truncation (to max. length)
    return_tensors='pt' # return PyTorch tensor
)

# transfer to device
# model_input = model_input.to(model.device)
# model_input = {key: tensor.to(base_model.device) for key, tensor in model_input.items()}

print(model_input)

In [ ]:
# print tokens
for input_ids in model_input['input_ids']:
    print(base_tokenizer.convert_ids_to_tokens(input_ids))

In [ ]:
# compute embeddings
base_out = base_model(**model_input)
last_hidden_state = base_out.last_hidden_state # (batch, sequence, features)

print(f'Embeddings shape: {last_hidden_state.shape}')

## Load classifier

In [ ]:
# set model name
seqclass_model_name = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

In [ ]:
# create tokenizer
seqclass_tokenizer = AutoTokenizer.from_pretrained(seqclass_model_name)

print(seqclass_tokenizer)

In [ ]:
# load model
seqclass_model = AutoModelForSequenceClassification.from_pretrained(
    seqclass_model_name,
    device_map='cpu',
    torch_dtype=torch.bfloat16 # use brain floating point format
)

seqclass_model = seqclass_model.eval()

print('Model device: {}'.format(seqclass_model.device))
print('Model dtype: {}'.format(seqclass_model.dtype))
print('Memory footprint: {:.2f} GiB'.format(seqclass_model.get_memory_footprint() * 1e-9))

print(seqclass_model)

## Classify sequences

In [ ]:
# create raw input
raw_input = [
    'This is great.',
    'This is awful.'
]

# tokenize
model_input = seqclass_tokenizer(
    raw_input,
    add_special_tokens=False, # add CLS token at the beginning
    padding=True, # turn on padding (for batched inputs)
    truncation=True, # turn on truncation (to max. length)
    return_tensors='pt' # return PyTorch tensor
)

# transfer to device
# model_input = model_input.to(model.device)
# model_input = {key: tensor.to(seqclass_model.device) for key, tensor in model_input.items()}

print(model_input)

In [ ]:
# print tokens
for input_ids in model_input['input_ids']:
    print(seqclass_tokenizer.convert_ids_to_tokens(input_ids))

In [ ]:
# predict logits
seqclass_out = seqclass_model(**model_input)
logits = seqclass_out.logits # (batch, sequence, labels)

print(f'Logits shape: {logits.shape}')

In [ ]:
# get predicted labels
label_ids = logits.argmax(dim=-1)
labels = [seqclass_model.config.id2label[lidx.item()] for lidx in label_ids]

print(labels)